### 1 - Import libs

**ONLY EXECUTE THIS CELL**

In [ ]:
import time 
from pathlib import Path
from datetime import datetime as dt

import ee
from sepal_ui.mapping import SepalMap
import geemap
# initialize EE    
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

import helpers as h

installed = !pip freeze | grep "retry=="
if not installed:
    !pip install --user retry

Styles()

### 2 - Basic Input Variables

**FILL IN YOUR INPUTS**

In [ ]:
# Area of Interest
import ee
import geemap

# country case example
#country = 'Kenya'   
#aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM0_NAME', country)) # here any feature collection can be selected

#aoi = ee.FeatureCollection("users/Erith7/Sucumbios")

# Point Feature Collection from where to extract points
#fc = ee.FeatureCollection("my_featurecollection_path").filterBounds(aoi)
#point_id_name = "PLOTID"  # the column/property within your point feature collection from which to take the UNIQUE point id

#fc = ee.FeatureCollection("users/impressfao/centre_2k_grid").filterBounds(aoi)
#fc = ee.FeatureCollection("users/Erith7/PuntosSucumbios").filterBounds(aoi)
aoi = ee.FeatureCollection('users/Erith7/Colombia2021/PilotoBfast1Colombia');
#point_id_name = "bb_tileID"
point_id_name = "Index"
print(fc.size().getInfo())

grid_size = 0.01  # that's the size of the grid we are parallelizing on
workers = 10 # number of parallel EE requests

# Time of interest (onsidering the historical period)
start_date = "2012-01-01"
end_date = "2021-12-31"

# Monitoring period
start_monitor = "2016-01-01"

# Directory where output and temp files will go
outdir = 'resultsMAATE'

# Select algorithms to run
cusum_deforest = True
bfast_monitor = True
bs_slope = True
ts_metrics = True
ccdc = False
global_products = False

band='ndvi' # other choices: ndfi, ndmi, mndwi

### DO NOT CHANGE YET ###
satellite='Landsat'

In [ ]:
# Grid Size (in degrees)
#grid_size = 0.1
# Generate the grid
grid, grid_fc = h.generate_grid(aoi, grid_size, grid_size, 0, 0)

# get a random point over each grid cell
randomInCell = grid_fc.map(h.get_random_point).filterBounds(aoi)

# get the centroid of each gridcell
centerInCell = grid_fc.map(h.get_center_point).filterBounds(aoi)

# stratified sampling based on ESA LC
imageCollection = ee.ImageCollection("ESA/WorldCover/v100")
esaLc = imageCollection.filterBounds(aoi).first().clip(aoi)

stratifiedSamples = esaLc.stratifiedSample(**{
    "numPoints": 100,
    "region": aoi,
    "scale": 100, 
    "seed": 42, 
    "tileScale": 4, 
    "geometries": True
}).map(h.set_id)

In [ ]:
# intialize Map
Map = SepalMap(['HYBRID'])
Map.zoom_ee_object(aoi.geometry())

# add layers
Map.addLayer(ee.Image().byte().paint(featureCollection=aoi,color=1,width=3), {"palette": "blue"}, 'Area of interest')
Map.addLayer(esaLc, False, 'ESA Worldcover 2020')
Map.addLayer(stratifiedSamples.draw(color="black", pointRadius=2), {}, 'Stratified Samples per class from ESA LC')
Map.addLayer(ee.Image().byte().paint(featureCollection=grid,color=1,width=3), {"palette": "black"}, 'Grid')
Map.addLayer(randomInCell.draw(color="purple", pointRadius=2), {}, '1 Center Sample per gridcell (Systematic)')
Map.addLayer(centerInCell.draw(color="white", pointRadius=2), {}, '1 Random Sample per gridcell (Systematic random)')
#
Map

### 3- Algorithm parameter settings

**Edit for advanced users, otherwise just execute**

In [5]:
# bfast parameters
bfast_params = {
    'run': bfast_monitor,
    'start_monitor': start_monitor, 
    'freq': 365,
    'k': 3, 
    'hfrac':0.25, 
    'trend': True, 
    'level':0.05, 
    'backend':'python'}

cusum_params = {
    'run': cusum_deforest,
    'nr_of_bootstraps': 1000
}

bs_slope_params = {
    'run': bs_slope,
    'nr_of_bootstraps': 1000
}

ts_metrics_params = {
    'run': ts_metrics,
    'outlier_removal': True,
    'z_threshhold': 3
}

ccdc_params = {
    'run': ccdc,
}

global_products = {
    'run': global_products,
}

### DO NOT CHANGE ###
### GATHER ALL INFO INTO A DICT #####
config_dict = {
    'work_dir': outdir,
    'workers': 10,
    'ts_params': {
        'start_date': start_date,
        'start_monitor': start_monitor,
        'end_date': end_date,
        'point_id': point_id_name,
        'grid_size': grid_size,
        'band': band,
        'satellite': satellite
    },
    'bfast_params': bfast_params,
    'cusum_params': cusum_params,
    'bs_slope_params': bs_slope_params,
    'ts_metrics_params': ts_metrics_params,
    'ccdc_params': ccdc_params,
    'global_products': global_products
}

# Run the whole thing

**Execute only**

In [6]:
h.get_change_data(aoi, stratifiedSamples, config_dict)

 You changed grid size. For this all temporary results are now being deleted.
 Parallelizing time-series extraction on 10 threads for a total of 397 grid cells.
 Grid cell 6 already has been extracted. Going on with next grid cell.
 Grid cell 4 already has been extracted. Going on with next grid cell.
 Grid cell 5 already has been extracted. Going on with next grid cell.
 Grid cell 7 already has been extracted. Going on with next grid cell.
 Grid cell 0 already has been extracted. Going on with next grid cell.
 Grid cell 9 already has been extracted. Going on with next grid cell.
 Grid cell 8 already has been extracted. Going on with next grid cell.
 Grid cell 2 already has been extracted. Going on with next grid cell.
 Grid cell 3 already has been extracted. Going on with next grid cell.
 Grid cell 1 already has been extracted. Going on with next grid cell.
 Grid cell 12 already has been extracted. Going on with next grid cell.
 Grid cell 13 already has been extracted. Going on with n

/usr/local/lib/python3.8/dist-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


 Processing has been finished successfully. Check for final_results files in your output directory.
